## Retriever Custom

### Creazione Vector-Store

In [2]:
# lettura di tutti i documenti da una cartella

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('data/dialoghi', glob="**/*.txt")
documents = loader.load()

for doc in documents:
    print(doc, "\n\n")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


page_content='Al negozio di abbigliamento

Commesso – Buongiorno, posso aiutarla? Cliente – Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli? Commesso – Certo, che taglia? Cliente – Porto una 50. Commesso – Eccoli qua!

Cliente – Grazie… però prima, posso dire che finalmente oggi si respira un po’? Ieri sembrava luglio! Commesso – Ah, guardi, non me ne parli. Stamattina ho già cambiato due volte la maglietta, da quanto fa caldo. E poi ieri sera quell’umidità… sembrava di stare in una serra. Cliente – Già, e poi è arrivato quel temporale all’improvviso, proprio mentre tornavo a casa. Mi sono bagnato fino alle ossa. Commesso – Eh sì, classico. Qui appena arriva un po’ di caldo, dopo due giorni parte il temporale estivo. Però oggi sembra tenere, almeno per ora. Cliente – Speriamo, che devo ancora fare un giro in centro dopo.

Commesso – A proposito, è appassionato di sport? Ieri notte ho guardato la partita dei Lakers contro i Celtics, non riuscivo a dormire! Clie

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=documents, embedding=OpenAIEmbeddings())

In [4]:
for doc in vectorstore.similarity_search("mamma", k=3):
    print(doc.metadata["source"], doc.page_content[:50] + "...\n\n")

data\dialoghi\dialogo3.txt Davanti alla scuola Mamma 1 - Ciao ragazze! Come s...


data\dialoghi\dialogo4.txt Al supermercato Cliente 1 - Ciao, scusa, sai dove ...


data\dialoghi\dialogo.txt Al negozio di abbigliamento

Commesso – Buongiorno...




In [5]:
retriever = vectorstore.as_retriever()
res = retriever.invoke("in quale dialogo ci sono delle mamme che parlano?")

for r in res:
    print(r.page_content.replace("\n", "")[:25], "\t", r.metadata)

Davanti alla scuola Mamma 	 {'source': 'data\\dialoghi\\dialogo3.txt'}
Al supermercato Cliente 1 	 {'source': 'data\\dialoghi\\dialogo4.txt'}
Al negozio di abbigliamen 	 {'source': 'data\\dialoghi\\dialogo.txt'}
In ufficio, durante la pa 	 {'source': 'data\\dialoghi\\dialogo5.txt'}


In [6]:
res[0]

Document(metadata={'source': 'data\\dialoghi\\dialogo3.txt'}, page_content="Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.\n\nMamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.\n\nMamma 2 - Avete sentito della nuova maestra di inglese? Dicono che s

# Retriever Custom
### tramite decoratore *chain*

Spesso è utile creare retriever custom per poter inserire ulteriori informazioni oltre a quelle direttamente fornite dal VectorStore.

In [7]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

res = retriever.invoke("in quale dialogo ci sono delle mamme che parlano?")

for r in res:
    print(r.page_content.replace("\n", "")[:25], "\t", r.metadata)

Davanti alla scuola Mamma 	 {'source': 'data\\dialoghi\\dialogo3.txt', 'score': 0.32311752438545227}
Al supermercato Cliente 1 	 {'source': 'data\\dialoghi\\dialogo4.txt', 'score': 0.44472822546958923}
Al negozio di abbigliamen 	 {'source': 'data\\dialoghi\\dialogo.txt', 'score': 0.4618980884552002}
In ufficio, durante la pa 	 {'source': 'data\\dialoghi\\dialogo5.txt', 'score': 0.4645719528198242}


In [8]:
print(res[0])

page_content='Davanti alla scuola Mamma 1 - Ciao ragazze! Come state oggi? Mamma 2 - Ciao! Bene, grazie. Solo un po' stanca, è stata una settimana lunga. E voi? Mamma 3 - Anche io sono un po' stanca. Tra lavoro e figli, non ho un attimo di pausa! Mamma 4 - Vi capisco perfettamente. Ogni tanto vorrei una giornata solo per me.

Mamma 1 - Sì, sarebbe fantastico! I vostri bambini come stanno andando a scuola? Mamma 2 - Marco è un po' in difficoltà con la matematica, ma stiamo lavorando con un tutor. Speriamo migliori presto. Mamma 3 - Laura adora la matematica, invece. Ma ha qualche problema con la storia. La trova noiosa. Mamma 4 - Anche Emma. Dice che le date sono troppe da ricordare. Ma almeno va bene in italiano. Mamma 1 - Pietro è bravo in tutte le materie, ma è molto disordinato. Trova sempre difficile tenere in ordine il suo zaino e i suoi quaderni.

Mamma 2 - Avete sentito della nuova maestra di inglese? Dicono che sia molto brava. Mamma 3 - Sì, mia figlia ne è entusiasta! Finalmen